In [44]:
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
from urllib.parse import urlparse
import os
import numpy as np
from IPython.display import clear_output, display
import time
import pandas as pd

from Constants import Constants, specialTokenList, specialTokens
from All_Models import SSCL, GatedCNN, SelfAttnModel
from utils import getSampler
from Trainer import Trainer
from LoadData import loadingData
import tkinter as tk


class args(object):
    init = ""


window = tk.Tk()
window.title('Spammer Detection')
window.geometry('1920x1280')


def setBoolHyerParams(window, var, text, x, y):
    c = tk.Checkbutton(window, text=text,
                        onvalue=True, offvalue=False, variable=var)
    c.place(x=x, y=y, anchor='nw')


def setHyperPrams(window, text, x, y):
    l = tk.Label(window,
                 text=text,    # 标签的文字
                 font=('Arial', 12),     # 字体和字体大小 # 标签长宽
                 )
    l.place(x=x, y=y, anchor='nw')
    e = tk.Entry(window)
    e.place(x=(x+200), y=y, anchor="nw")
    return l, e


def setRadioButtons(var, label, texts, values, command, x, y):

    l = tk.Label(window,
                 text=label,  
                 font=('Arial', 12),   
                 )
    l.place(x=x, y=y, anchor='nw')

    for t, v in zip(texts, values):
        y = y+40
        tk.Radiobutton(window, text=t, font=('Arial', 12),
                       variable=var, value=v,
                       command=command,).place(x=x, y=y, anchor='nw')


############### Dataset ###############

def Take_DatasetSelection():
    print("Selected Dataset: ", datasetSelected.get())


datasetSelected = tk.StringVar()

setRadioButtons(datasetSelected, "Dataset:",
                ["HSpam14Dataset", "HoneypotDataset"], ['HSpam14', 'Honeypot'],
                Take_DatasetSelection, 10, 10)


############### Models ###############


def Take_ModelSelection():
    printHyperParamsSetting(window, modelSelected.get(), 250, 10)


modelSelected = tk.StringVar()

models_list = ['SSCL', 'GatedCNN', 'SelfAttn']

setRadioButtons(modelSelected, "Model:", models_list,
                models_list, Take_ModelSelection, 10, 150)


x = 250
y = 10


GatedCNN_embedingDim_label, GatedCNN_embedingDim = setHyperPrams(
    window, "GatedCNN_embedingDim", x, y)
GatedCNN_convDim_label, GatedCNN_convDim = setHyperPrams(
    window, "GatedCNN_convDim", x, y+(30*1))
GatedCNN_kernel_label, GatedCNN_kernel = setHyperPrams(
    window, "GatedCNN_kernel", x, y+(30*2))
GatedCNN_stride_label, GatedCNN_stride = setHyperPrams(
    window, "GatedCNN_stride", x, y+(30*3))
GatedCNN_pad_label, GatedCNN_pad = setHyperPrams(
    window, "GatedCNN_pad", x, y+(30*4))
GatedCNN_layers_label, GatedCNN_layers = setHyperPrams(
    window, "GatedCNN_layers", x, y+(30*5))
GatedCNN_dropout_label, GatedCNN_dropout = setHyperPrams(
    window, "GatedCNN_dropout", x, y+(30*6))

SSCL_RNNHidden_label, SSCL_RNNHidden = setHyperPrams(
    window, "SSCL_RNNHidden", x, y)
SSCL_CNNDim_label, SSCL_CNNDim = setHyperPrams(
    window, "SSCL_CNNDim", x, y+(30*1))
SSCL_CNNKernel_label, SSCL_CNNKernel = setHyperPrams(
    window, "SSCL_CNNKernel", x, y+(30*2))
SSCL_CNNDropout_label, SSCL_CNNDropout = setHyperPrams(
    window, "SSCL_CNNDropout", x, y+(30*3))
SSCL_LSTMDropout_label, SSCL_LSTMDropout = setHyperPrams(
    window, "SSCL_LSTMDropout", x, y+(30*4))
SSCL_LSTMLayers_label, SSCL_LSTMLayers = setHyperPrams(
    window, "SSCL_LSTMLayers", x, y+(30*5))
SSCL_embedingDim_label, SSCL_embedingDim = setHyperPrams(
    window, "SSCL_embedingDim", x, y+(30*6))

SelfAttn_LenMaxSeq_label, SelfAttn_LenMaxSeq = setHyperPrams(
    window, "SelfAttn_LenMaxSeq", x, y)
SelfAttn_ModelDim_label, SelfAttn_ModelDim = setHyperPrams(
    window, "SelfAttn_ModelDim", x, y+(30*1))
SelfAttn_FFInnerDim_label, SelfAttn_FFInnerDim = setHyperPrams(
    window, "SelfAttn_FFInnerDim", x, y+(30*2))
SelfAttn_NumLayers_label, SelfAttn_NumLayers = setHyperPrams(
    window, "SelfAttn_NumLayers", x, y+(30*3))
SelfAttn_NumHead_label, SelfAttn_NumHead = setHyperPrams(
    window, "SelfAttn_NumHead", x, y+(30*4))
SelfAttn_KDim_label, SelfAttn_KDim = setHyperPrams(window, "SelfAttn_KDim", x, y+(30*5))
SelfAttn_VDim_label, SelfAttn_VDim = setHyperPrams(window, "SelfAttn_VDim", x, y+(30*6))
SelfAttn_Dropout_label, SelfAttn_Dropout = setHyperPrams(
    window, "SelfAttn_Dropout", x, y+(30*7))

GatedCNN_label = [
    GatedCNN_embedingDim_label,
    GatedCNN_convDim_label,
    GatedCNN_kernel_label,
    GatedCNN_stride_label,
    GatedCNN_pad_label,
    GatedCNN_layers_label,
    GatedCNN_dropout_label,
]

GatedCNN_entry = [
    GatedCNN_embedingDim,
    GatedCNN_convDim,
    GatedCNN_kernel,
    GatedCNN_stride,
    GatedCNN_pad,
    GatedCNN_layers,
    GatedCNN_dropout,
]

SSCL_label = [
    SSCL_RNNHidden_label,
    SSCL_CNNDim_label,
    SSCL_CNNKernel_label,
    SSCL_CNNDropout_label,
    SSCL_LSTMDropout_label,
    SSCL_LSTMLayers_label,
    SSCL_embedingDim_label
]

SSCL_entry = [
    SSCL_RNNHidden,
    SSCL_CNNDim,
    SSCL_CNNKernel,
    SSCL_CNNDropout,
    SSCL_LSTMDropout,
    SSCL_LSTMLayers,
    SSCL_embedingDim,
]


SelfAttn_label = [
    SelfAttn_LenMaxSeq_label,
    SelfAttn_ModelDim_label,
    SelfAttn_FFInnerDim_label,
    SelfAttn_NumLayers_label,
    SelfAttn_NumHead_label,
    SelfAttn_KDim_label,
    SelfAttn_VDim_label,
    SelfAttn_Dropout_label,
]

SelfAttn_entry = [
    
    SelfAttn_LenMaxSeq,
    SelfAttn_ModelDim,
    SelfAttn_FFInnerDim,
    SelfAttn_NumLayers,
    SelfAttn_NumHead,
    SelfAttn_KDim,
    SelfAttn_VDim,
    SelfAttn_Dropout,
]


def forgetLabelAndEntry(label, entry):
    for l, e in zip(label, entry):
        l.place_forget()
        e.place_forget()


forgetLabelAndEntry(SSCL_label, SSCL_entry)
forgetLabelAndEntry(SelfAttn_label, SelfAttn_entry)
forgetLabelAndEntry(GatedCNN_label, GatedCNN_entry)

def placeLabelAndEntry(labels, entries, x, y):
    for l, e in zip(labels, entries):
        y = y + 30
        l.place(x=x, y=y, anchor='nw')
        e.place(x=(x+250), y=y, anchor="nw")


def printHyperParamsSetting(window, model, x, y):



    x = 250
    y = 10

    if model == 'GatedCNN':

        forgetLabelAndEntry(SSCL_label, SSCL_entry)
        forgetLabelAndEntry(SelfAttn_label, SelfAttn_entry)
        placeLabelAndEntry(GatedCNN_label, GatedCNN_entry, x, y)

    elif model == "SSCL":

        # Have to fix this part in the SSCL Model

        forgetLabelAndEntry(GatedCNN_label, GatedCNN_entry)
        forgetLabelAndEntry(SelfAttn_label, SelfAttn_entry)
        placeLabelAndEntry(SSCL_label, SSCL_entry, x, y)

    elif model == "SelfAttn":

        forgetLabelAndEntry(GatedCNN_label, GatedCNN_entry)
        forgetLabelAndEntry(SSCL_label, SSCL_entry)
        placeLabelAndEntry(SelfAttn_label, SelfAttn_entry, x, y)

    else:
        raise ValueError

        
        
x = 770 
y = 30

vocab_size_label, vocab_size = setHyperPrams(window, "vocab_size", x, y)
validation_portion_label, validation_portion = setHyperPrams(window, "validation_portion", x, (y+(30*1)))
test_portion_label, test_portion = setHyperPrams(window, "test_portion", x, (y+(30*2)))
batch_size_label, batch_size = setHyperPrams(window, "batch_size", x, (y+(30*3)))
L2_label, L2 = setHyperPrams(window, "L2", x, (y+(30*4)))
lr_label, lr = setHyperPrams(window, "lr", x, (y+(30*5)))
n_epoch_label, n_epoch = setHyperPrams(window, "n_epoch", x, (y+(30*6)))
earlyStopStep_label, earlyStopStep = setHyperPrams(window, "earlyStopStep", x, (y+(30*7)))
earlyStopEpoch_label, earlyStopEpoch = setHyperPrams(window, "earlyStopEpoch", x, (y+(30*8)))
val_freq_label, val_freq = setHyperPrams(window, "val_freq", x, (y+(30*9)))
val_steps_label, val_steps = setHyperPrams(window, "val_steps", x, (y+(30*10)))
log_freq_label, log_freq = setHyperPrams(window, "log_freq", x, (y+(30*11)))
model_name_label, model_name = setHyperPrams(window, "model_name", x, (y+(30*12)))

usingWeightRandomSampling = tk.StringVar()
setBoolHyerParams(window, usingWeightRandomSampling, "usingWeightRandomSampling", x, (y+(30*14)))


vocab_size.insert("end", 8000)
validation_portion.insert("end", 0.05)
test_portion.insert("end", 0.04)
batch_size.insert("end", 64)
L2.insert("end", 0.1)
lr.insert("end", 0.002)
n_epoch.insert("end", 50)
earlyStopStep.insert("end", 50000)
earlyStopEpoch.insert("end", 1)
val_freq.insert("end", 50)
val_steps.insert("end", 3)
log_freq.insert("end", 10)



GatedCNN_embedingDim.insert("end",128)
GatedCNN_convDim.insert("end",64)
GatedCNN_kernel.insert("end",3)
GatedCNN_stride.insert("end",1)
GatedCNN_pad.insert("end",1)
GatedCNN_layers.insert("end",8)
GatedCNN_dropout.insert("end",0.1)
SSCL_embedingDim.insert("end", 512)            
SSCL_RNNHidden.insert("end",256)
SSCL_CNNDim.insert("end",64)
SSCL_CNNKernel.insert("end",5)
SSCL_CNNDropout.insert("end",0.1)
SSCL_LSTMDropout.insert("end",0.1)
SSCL_LSTMLayers.insert("end",1)
SelfAttn_LenMaxSeq.insert("end", 280)
SelfAttn_ModelDim.insert("end", 128)
SelfAttn_FFInnerDim.insert("end",256)
SelfAttn_NumLayers.insert("end",3)
SelfAttn_NumHead.insert("end",4)
SelfAttn_KDim.insert("end",64)
SelfAttn_VDim.insert("end",64)
SelfAttn_Dropout.insert("end",0.1)




def StartTraining():
    print("Training Start")

b = tk.Button(window, 
    text='Strat Training',     
    width=15, height=2, 
    command=StartTraining)     
b.place(x=1000, y=500, anchor="nw") 









window.mainloop()


Training Start
Training Start


In [ ]:
GatedCNN_label = [
    GatedCNN_embedingDim_label,
    GatedCNN_convDim_label,
    GatedCNN_kernel_label,
    GatedCNN_stride_label,
    GatedCNN_pad_label,
    GatedCNN_layers_label,
    GatedCNN_dropout_label,
]

GatedCNN_entry = [
    GatedCNN_embedingDim,
    GatedCNN_convDim,
    GatedCNN_kernel,
    GatedCNN_stride,
    GatedCNN_pad,
    GatedCNN_layers,
    GatedCNN_dropout,
]

SSCL_label = [
    SSCL_RNNHidden_label,
    SSCL_CNNDim_label,
    SSCL_CNNKernel_label,
    SSCL_CNNDropout_label,
    SSCL_LSTMDropout_label,
    SSCL_LSTMLayers_label,

]

SSCL_entry = [
    
    SSCL_RNNHidden,
    SSCL_CNNDim,
    SSCL_CNNKernel,
    SSCL_CNNDropout,
    SSCL_LSTMDropout,
    SSCL_LSTMLayers,
]


SelfAttn_label = [
    SelfAttnLenMaxSeq_label,
    SelfAttnModelDim_label,
    SelfAttnFFInnerDim_label,
    SelfAttnNumLayers_label,
    SelfAttnNumHead_label,
    SelfAttnKDim_label,
    SelfAttnVDim_label,
    SelfAttnDropout_label,
]

SelfAttn_entry = [
    
    SelfAttnLenMaxSeq,
    SelfAttnModelDim,
    SelfAttnFFInnerDim,
    SelfAttnNumLayers,
    SelfAttnNumHead,
    SelfAttnKDim,
    SelfAttnVDim,
    SelfAttnDropout,
]

In [ ]:
















var =  tk.StringVar() 

l = tk.Label(window, 
    textvariable=var,    # 标签的文字
    font=('Arial', 12),     # 字体和字体大小
    width=15, height=2  # 标签长宽
    )
l.pack()    # 固定窗口位置


l2 = tk.Label(window, 
    text='Text Data!',    # 标签的文字
    font=('Arial', 12),     # 字体和字体大小
    width=15, height=2  # 标签长宽
    )
l2.pack()    

b = tk.Button(window, 
    text='hit me',      # 显示在按钮上的文字
    width=15, height=2, 
    command=hit_me)     # 点击按钮式执行的命令
b.pack()    # 按钮位置




on_hit = False  # 默认初始状态为 False
def hit_me():
    global on_hit
    if on_hit == False:     # 从 False 状态变成 True 状态
        on_hit = True
        var.set('you hit me')   # 设置标签的文字为 'you hit me'
    else:       # 从 True 状态变成 False 状态
        on_hit = False
        var.set('') # 设置 文字为空


        
''' 
For Taking Input
'''        

e = tk.Entry(window)
# e = tk.Entry(window,show='*')
e.pack()
        
t = tk.Text(window,height=2)
t.pack()
        
    
var1 = tk.StringVar()    #创建变量
l =tk.Label(window,bg='yellow',width=4,textvariable=var1)
l.pack()
    
def print_selection():
    value = lb.get(lb.curselection())   #获取当前选中的文本
    var.set(value)     #为label设置值
    
    
'''
List
'''

b1 = tk.Button(window, text='print selection', width=15,
              height=2, command=print_selection)
b1.pack()
    
var2 = tk.StringVar()
var2.set((11,22,33,44)) #为变量设置值

#创建Listbox

lb = tk.Listbox(window, listvariable=var2)  #将var2的值赋给Listbox



#创建一个list并将值循环添加到Listbox控件中
# list_items = [1,2,3,4]
# for item in list_items:
#     lb.insert('end', item)  #从最后一个位置开始加入值
# lb.insert(1, 'first')       #在第一个位置加入'first'字符
# lb.insert(2, 'second')      #在第二个位置加入'second'字符
# lb.delete(2)                #删除第二个位置的字符
lb.pack()

var = tk.StringVar()
l = tk.Label(window, bg='yellow', width=20, text='empty')
l.pack()

r1 = tk.Radiobutton(window, text='Option A',
                    variable=var, value='A',
                    command=print_selection)
r1.pack()

def print_selection():
    l.config(text='you have selected ' + var.get())